# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os
import pickle
from sqlalchemy import create_engine
import re
import nltk
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from scipy.stats import hmean
from scipy.stats.mstats import gmean
from custom_transformer import StartingVerbExtractor
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# load data from database
cwd = os.getcwd()
dbwd = cwd.replace('/models','/data/').replace('\\models','\\data\\')

engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('df', engine) #read_sql_table Read SQL database table into a DataFrame.
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [4]:
def tokenize(text):
    # get list of all urls using regex
    detected_urls =  re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder") #

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

**Each of these objects is called an estimator.The main API implemented by scikit-learn is that of the estimator.An estimator is any object that learns from data;it may be a classification, regrission or clustering algorthm or a transformer that extract filter useful features from raw data.**



In [ ]:

#to creat this pipeline we just need a list of key value pairs
    
# each of tese three estimators have a fit method. 
# additionally CountVectorizer and TfidfTransformer are specific type of estimator called a transformer ;meaning it has a transformer method 
# the final estimator we have here is a predictor. which although it doesn't have a transform method but has a predict method 



###Note 
   #that every step of a pipeline has to be a transformer, 
   #except for the last step which can be an estimator of any type.   
   #pipeline takes on all the methods of whatever the last estimator in its sequence in.
   

### Feature unions 

**Feature union is a class in scikit-learn’s Pipeline module that allows us to perform steps in parallel and take the union of their results for the next step**

**Feature unions are super helpful for handling these situations, where we need to run two steps in parallel on the same data and combine their results to pass into the next step. Like pipelines, feature unions are built using a list of (key, value) pairs. Also like pipelines, feature unions combine a list of estimators to become a single estimators. Notice we use a pipeline in this feature union to make sure the count vectorizer and tfidf transformer steps are still running in sequence**

In [5]:
### with Feature Union
#Using the given custom transformer, StartingVerbExtractor, add a feature union to your pipeline to incorporate a feature that indicates with a boolean value whether the starting token of a post is identified as a verb.

def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

#we use a pipeline in Feature Union 
#to make sure "CountVectorizer" and "TfidfTransformer" steps are still running in sequence 
            
            ('text_pipeline', Pipeline([                 #'text_pipeline'  where the key is the string that you want to name a step,
                ('vect', CountVectorizer(tokenizer=tokenize)),  ## and the value is the estimator object Pipeline , CountVectorizer, TfidfTransformer, StartingVerbExtractor
                ('tfidf', TfidfTransformer())
            ])),
            
            ('starting_verb', StartingVerbExtractor()) #we used a custom_transformer that extracted whether each text started with a verb
        ])),
#also similary a feature union combines alist of estimators to become a single estimator
#it runs its estimators in parallel, rather than in a sequence as a pipeline does.
#the estimators being run in parallel are 'text_pipeline' and 'starting_verb', we need both the outputs to feed into the classifier as input 
        
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

model = []
model = model_pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = model.predict(X_test)

In [8]:
def multioutput_fscore(y_true,y_pred,beta=1):
    score_list = []
    if isinstance(y_pred, pd.DataFrame) == True:
        y_pred = y_pred.values
    if isinstance(y_true, pd.DataFrame) == True:
        y_true = y_true.values
    for column in range(0,y_true.shape[1]):
        score = fbeta_score(y_true[:,column],y_pred[:,column],beta,average='weighted')
        score_list.append(score)
    f1score_numpy = np.asarray(score_list)
    f1score_numpy = f1score_numpy[f1score_numpy<1]
    f1score = gmean(f1score_numpy)
    return  f1score

In [9]:
multi_f1 = multioutput_fscore(y_test,y_pred, beta = 1)
overall_accuracy = (y_pred == y_test).mean().mean()



In [10]:
print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score (custom definition) {0:.2f}%\n'.format(multi_f1*100))

Average overall accuracy 94.87% 

F1 score (custom definition) 93.80%



In [11]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))


------------------------------------------------------

FEATURE: related

             precision    recall  f1-score   support

          0       0.58      0.28      0.38      1581
          1       0.80      0.93      0.86      4935
          2       0.35      0.16      0.22        38

avg / total       0.74      0.77      0.74      6554

------------------------------------------------------

FEATURE: request

             precision    recall  f1-score   support

          0       0.92      0.96      0.94      5476
          1       0.75      0.55      0.63      1078

avg / total       0.89      0.90      0.89      6554

------------------------------------------------------

FEATURE: offer

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

------------------------------------------------------

FEATURE: aid_related

    

### 6. Improve your model
Use grid search to find better parameters.

**most machine learning algorithm have a set of parameter that need tuning.grid search is a tool that allows us to define a grid of parameters or a set of values to check**

**make sure you run standard scaler only on the training set, and not the validation set within each fold of cross validation.**

**Pipelines allow you to do just this.since the re-scaling is included as part of the pipeline, the standardization doesn't happen until we run the grid scearch**

**meaning in each fold of cross validation, rescaling is only the data that the model is being trained on preventing leakage from the validation set**

**As you can see, pipelines are very valuable to removing the risk of data leakage during the data preparation process.**

In [23]:
def build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])
#All you need to do is create a dictionary of parameters to search, 
#using keys for the names of the parameters and values for the list of parameter values to check. 
 
    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
       
    }


    scorer = make_scorer(multioutput_fscore,greater_is_better = True)

    cv = GridSearchCV(model, param_grid=parameters, scoring = scorer,verbose = 2, n_jobs = -1) ##Then, pass the model and parameter grid to the grid search object.
    
    return cv

In [24]:
#Now when you call fit on this grid search object, 
#it will run cross validation on all different combinations of these parameters to find the best combination of parameters for the model.
build_model().fit(X_train, y_train)


Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1) 
[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1), total= 2.5min
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1) 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.2min remaining:    0.0s


[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1), total= 2.5min
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1) 
[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1), total= 2.5min
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1

[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed: 395.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)), 'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0), 'features__text_pipeline__vect__max_features': (None, 5000, 10000), 'features__text_pipeline__tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(multioutput_fscore), verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [25]:
filename = 'classifier.sav'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.